In [1]:
import pandas as pd
import numpy as np
import os 
from tqdm import tqdm
#importing libraries
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from os import path
from pydub import AudioSegment

In [2]:
df=pd.read_csv('list2.csv')
df

,Unnamed: 0,File,Language
0,0,171.mp3,Bengali
1,1,172.mp3,Bengali
2,2,173.mp3,Bengali
3,3,174.mp3,Bengali
4,4,175.mp3,Bengali
...,...,...,...
300,26,856.mp3,Telugu
301,27,862.mp3,Telugu
302,28,866.mp3,Telugu
303,29,908.mp3,Telugu


In [3]:
MFCC_NUM = 20
MFCC_MAX_LEN = 2000

In [4]:
def features_extractor(file,lang,max_len=2000):
    src = "./Data/Train/"+lang+"1/"+file
    dst = "./Data/test2.wav"
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="wav")
    audio, sample_rate = librosa.load(dst, mono=True,sr=44100) 
    audio=audio[::3]
    mfcc = librosa.feature.mfcc(y=audio, sr=44100, n_mfcc=20)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [5]:
extracted_features=[]
for index_num,row in tqdm(df.iterrows()):
    file_Name = str(row["File"])
    class_labels=row["Language"]
    data=features_extractor(file_Name,class_labels)
    extracted_features.append([data,class_labels])

305it [00:35,  8.48it/s]


In [6]:
extracted_features[:3]

[[array([[-5.2946625e+02, -4.5446738e+02, -3.9308878e+02, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [ 4.8457233e+01,  9.5342575e+01,  1.5136635e+02, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [ 2.8190834e+01,  2.5879881e+01,  3.9971344e+01, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         ...,
         [-1.0993437e+01, -5.0499296e+00,  7.1224375e+00, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [-6.9087219e+00,  9.1962814e-02,  7.1970024e+00, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [-5.3697634e+00, -3.7932420e-01,  9.5489740e-01, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32),
  'Bengali'],
 [array([[-5.4273145e+02, -4.4655005e+02, -2.6354449e+02, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [ 8.5125893e+01,  1.2557796e+02,  1.1585376e+02, ...,
           0.0000000e+00,  0.0000000e+

In [7]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['Vectors','Lang'])
extracted_features_df.head(5)

,Vectors,Lang
0,"[[-529.46625, -454.46738, -393.08878, -297.311...",Bengali
1,"[[-542.73145, -446.55005, -263.5445, -161.7895...",Bengali
2,"[[-580.6701, -480.8971, -398.3205, -236.64789,...",Bengali
3,"[[-423.3275, -332.37094, -332.72223, -315.9054...",Bengali
4,"[[-247.7952, -113.57214, -67.81812, -92.65014,...",Bengali


In [8]:
shuffled = extracted_features_df.sample(frac=1)
print(shuffled)

                                               Vectors      Lang
3    [[-423.3275, -332.37094, -332.72223, -315.9054...   Bengali
80   [[-588.9912, -516.06537, -523.1096, -582.8048,...     Hindi
182  [[-360.6193, -304.20007, -299.7313, -230.34013...      Urdu
180  [[-308.90338, -228.43907, -251.75803, -167.114...      Urdu
148  [[-417.26654, -245.93082, -193.48895, -194.758...   Punjabi
..                                                 ...       ...
62   [[-613.6033, -435.60806, -227.61128, -173.5785...  Gujrathi
13   [[-547.1625, -518.1402, -540.4688, -549.61835,...   Bengali
214  [[-366.7488, -270.12405, -282.13925, -298.8207...  Malyalam
268  [[-454.10962, -352.8136, -327.1522, -341.45334...   Marathi
168  [[-481.72382, -384.03067, -268.27115, -197.915...      Urdu

[305 rows x 2 columns]


In [9]:
#split into dependent and independent variables
X=np.array(shuffled['Vectors'].tolist())
y=np.array(shuffled['Lang'].tolist())

In [10]:
X.dtype

dtype('float32')

In [11]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

#Label encoding
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
y



array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [12]:
import tensorflow
import keras
import tensorflow.keras.utils
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

In [15]:
X_train.shape

(195, 20, 2000)

In [16]:
feature_dim_1 = MFCC_NUM
# Second dimension of the feature is dim2
feature_dim_2 = MFCC_MAX_LEN
channel = 1
epochs = 70
batch_size = 100
verbose = 1
num_classes = len(y[0])

In [17]:
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras import optimizers

In [19]:
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 19, 1999, 32)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 1998, 48)      6192      
                                                                 
 max_pooling2d (MaxPooling2  (None, 9, 999, 48)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 9, 999, 48)        0         
                                                                 
 flatten (Flatten)           (None, 431568)            0         
                                                                 
 dense (Dense)               (None, 128)               55240832  
                                                        

In [21]:
import tensorflow as tf

optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.002, decay=1e-6, momentum=0.9, nesterov=True)

In [22]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test))   


Epoch 1/70
2/2 [==============================] - 9s 4s/step - loss: 83.9281 - accuracy: 0.0769 - val_loss: 32.9877 - val_accuracy: 0.1148
Epoch 2/70
2/2 [==============================] - 6s 3s/step - loss: 30.1631 - accuracy: 0.0974 - val_loss: 2.3207 - val_accuracy: 0.0656
Epoch 3/70
2/2 [==============================] - 7s 3s/step - loss: 2.3141 - accuracy: 0.1026 - val_loss: 2.2979 - val_accuracy: 0.1311
Epoch 4/70
2/2 [==============================] - 6s 3s/step - loss: 2.2870 - accuracy: 0.0821 - val_loss: 2.2971 - val_accuracy: 0.0820
Epoch 5/70
2/2 [==============================] - 7s 4s/step - loss: 2.2721 - accuracy: 0.1487 - val_loss: 2.2961 - val_accuracy: 0.0984
Epoch 6/70
2/2 [==============================] - 7s 3s/step - loss: 2.2136 - accuracy: 0.2051 - val_loss: 2.2905 - val_accuracy: 0.1148
Epoch 7/70
2/2 [==============================] - 6s 3s/step - loss: 2.2260 - accuracy: 0.1590 - val_loss: 2.2929 - val_accuracy: 0.2131
Epoch 8/70
2/2 [======================

2/2 [==============================] - 16s 8s/step - loss: 0.2299 - accuracy: 0.9641 - val_loss: 2.0043 - val_accuracy: 0.4590
Epoch 61/70
2/2 [==============================] - 12s 4s/step - loss: 0.2613 - accuracy: 0.9333 - val_loss: 2.0482 - val_accuracy: 0.4262
Epoch 62/70
2/2 [==============================] - 12s 4s/step - loss: 0.2661 - accuracy: 0.9282 - val_loss: 2.4316 - val_accuracy: 0.3607
Epoch 63/70
2/2 [==============================] - 7s 4s/step - loss: 0.2287 - accuracy: 0.9179 - val_loss: 2.2035 - val_accuracy: 0.4098
Epoch 64/70
2/2 [==============================] - 7s 3s/step - loss: 0.1989 - accuracy: 0.9385 - val_loss: 2.1701 - val_accuracy: 0.4262
Epoch 65/70
2/2 [==============================] - 13s 4s/step - loss: 0.2369 - accuracy: 0.9231 - val_loss: 2.1298 - val_accuracy: 0.4098
Epoch 66/70
2/2 [==============================] - 7s 4s/step - loss: 0.2157 - accuracy: 0.9385 - val_loss: 2.0149 - val_accuracy: 0.4098
Epoch 67/70
2/2 [=========================

In [24]:
X_validation = X_validation.reshape(X_validation.shape[0], feature_dim_1, feature_dim_2, channel)

In [25]:
model.predict(X_validation)[0]

2/2 [==============================] - 1s 118ms/step


array([1.6473841e-05, 3.6644863e-07, 3.4251694e-05, 1.9439696e-07,
       2.6158762e-06, 2.8528734e-07, 6.5810550e-06, 9.9991977e-01,
       1.9429153e-05, 7.8207840e-09], dtype=float32)